In [1]:
# 显示cell运行时长
%load_ext klab-autotime

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import psutil
import gc

time: 6.89 s


In [3]:
app_type={'user_id':np.uint32,'day':np.uint8}
act_type={'user_id':np.uint32,'day':np.uint8,'page':np.uint8,'video_id':np.uint32,'author_id':np.uint32,'action_type':np.uint8}
reg_type = {'user_id':np.uint32,'register_day':np.uint8,'register_type':np.uint8,'device_type':np.uint8}
video_type={'user_id':np.uint32,'day':np.uint8}

time: 2.44 ms


In [4]:
path = "/mnt/datasets/fusai/"
app = pd.read_csv(path+"app_launch_log.txt",names=['user_id','day'],encoding='utf-8',dtype=app_type,sep="\t")
user_act = pd.read_table(path+'user_activity_log.txt',names=['user_id','day','page','video_id','author_id','action_type'],encoding='utf-8',dtype=act_type,sep='\t')
user_reg = pd.read_table(path+'user_register_log.txt',names=['user_id','register_day','register_type','device_type'],encoding='utf-8',dtype=reg_type,sep='\t')
video = pd.read_table(path+'video_create_log.txt',names=['user_id','day'],dtype=video_type,encoding='utf-8',sep='\t')

time: 1min 37s


In [5]:
def getMemCpu():
    data = psutil.virtual_memory()
    total = data.total #总内存,单位为byte
    free = data.available #可以内存
    memory =  "Memory usage:%d"%(int(round(data.percent)))+"%"+"  "
    cpu = "CPU:%0.2f"%psutil.cpu_percent(interval=1)+"%"
    return memory+cpu

time: 2.42 ms


In [6]:
def get_last_day_count(d,data,row_name):
    data = data[data.day>d]
    last_day_count = data[['user_id','day']].groupby('user_id')['day'].agg({(row_name+"days_counts",'count')}).reset_index()
    return last_day_count

time: 1.5 ms


In [7]:
#求最后一次与前一次的差
def get_last_gap(s):
    s = list(s)
    n = len(s)
    if n<2:
        return None
    s.sort()
    return s[n-1]-s[n-2]

time: 1.86 ms


In [8]:
#求连续行为天数的最大值
def get_continue_counts(data):
    data = np.array(data)
    i=0
    count = 1
    max_num = 0
    while i<len(data)-1:
        if data[i+1]-data[i] == 1:
            count += 1
            i += 1
        else:
            if max_num < count:
                max_num = count
            i += 1
            count = 1
    if max_num < count :
        max_num = count
    return max_num

time: 4.24 ms


In [9]:
#求连续行为天数的均值
def get_continue_mean(data):
    data = np.array(data)
    i=0
    count = 1
    count_arr = []
    while i<len(data)-1:
        if data[i+1]-data[i] == 1:
            count += 1
            i += 1
        else:
            count_arr.append(count)
            i += 1
            count = 1
    count_arr.append(count)
    return np.mean(count_arr)

time: 2.23 ms


In [10]:
#求连续行为天数的方差
def get_continue_std(data):
    data = np.array(data)
    i=0
    count = 1
    count_arr = []
    ans = []
    while i<len(data)-1:
        if data[i+1]-data[i] == 1:
            count += 1
            i += 1
        else:
            count_arr.append(count)
            i += 1
            count = 1
    count_arr.append(count)
    return np.std(count_arr)

time: 2.34 ms


In [18]:
def cal_kurt(data):
    return data.kurt()

time: 1.3 ms


In [19]:
#一阶差分后求特征
def diff_feature(row_name,data):
    diff_feature=[]
    if 'day' in data.columns.values.tolist():
        user_day='day' 
    else:
        user_day='register_day'
    diff_feature = data.sort_values(by=['user_id',user_day])

    diff_feature['diff']=data.groupby('user_id')[user_day].diff().fillna(0).astype("float32")
    diff_feature.drop(columns=[user_day],inplace=True)
    
    diff_fun = {(row_name+'sum','sum'),(row_name+'var','var'),(row_name+'mean','mean'),(row_name+'max','max'),
            (row_name+'std','std'),(row_name+'min','min'),(row_name+'count','count'),(row_name+'skew','skew'),
            (row_name+'kurt',cal_kurt)
    }
    diff = []
    diff = diff_feature.groupby('user_id')['diff'].agg(diff_fun).astype("float32").reset_index()
    return diff

time: 4.2 ms


In [20]:
#对时间求特征
def day_feature(row_name,data):
    day_feature =[]
    if 'day' in data.columns.values.tolist():
        user_day='day' 
    else:
        user_day='register_day'
        
    day_fun = {(row_name+'sum','sum'),(row_name+'var','var'),(row_name+'count','count'),(row_name+'std','std'),
                (row_name+'max','max'),(row_name+'mean','mean'),(row_name+'min','min'),(row_name+'skew','skew'),
                (row_name+'kurt',cal_kurt)
    }
    day_feature = data.groupby('user_id')[user_day].agg(day_fun).astype("float32").reset_index()
    return day_feature

time: 3.29 ms


In [15]:
def cal_feature_counts(row_name,data,c_name):
    
    cal_count=[]
    # data = data[data.day>7]
    cal_count = data.groupby(['user_id',c_name]).size().rename("column_counts").reset_index()
    newDF = pd.DataFrame()
    newDF['user_id'] = cal_count['user_id'].unique()
    days = list(data[c_name].unique())
    for c in days:
        col_name = '{}_count'.format(str(row_name) + str(c))
        cal_count = data[data[c_name]==c].groupby(['user_id']).size().astype("float32").rename(col_name).reset_index()
        newDF = pd.merge(newDF,cal_count,on=['user_id'],how='left')
    return newDF

time: 2.9 ms


In [21]:
def app_feature(beginday,endday):
    
    info = getMemCpu()
    start = time.clock()
    
    app_content = app[(app.day>=beginday)&(app.day<=endday)]
    app_content.day = app_content.day - beginday + 1
    
    app_feature = day_feature('app_day_',app_content)
    
    diff = diff_feature('app_day_diff_',app_content)
    app_feature = pd.merge(app_feature,diff,on=['user_id'],how='left')
    del diff
    gc.collect()
    
    app_nums=app_content.groupby('user_id')['day'].nunique().astype("float32").reset_index().rename(columns={'day':'app_nums'})
    app_feature = pd.merge(app_feature,app_nums,on=['user_id'],how='left')
    print("app_nums")
    del app_nums
    gc.collect()
    
    gp = app_content[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_last_gap).astype("float32").rename('app_last_gap').reset_index()
    app_feature = pd.merge(app_feature,gp,on=['user_id'],how='left')
    del gp
    gc.collect()
    
    app_continue_std = app_content[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_continue_std).rename('app_continue_std').astype("float32").reset_index()
    app_feature = pd.merge(app_feature,app_continue_std,on=['user_id'],how='left')
    del app_continue_std
    gc.collect()
    
    app_continue_mean = app_content[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_continue_mean).rename('app_continue_mean').astype("float32").reset_index()
    app_feature = pd.merge(app_feature,app_continue_mean,on=['user_id'],how='left')
    del app_continue_mean
    gc.collect()
    
    app_last1days_counts = get_last_day_count(15,app_content,"app_last1")
    app_feature = pd.merge(app_feature,app_last1days_counts,on=['user_id'],how='left')
    del app_last1days_counts
    gc.collect()
    
    app_last2days_counts = get_last_day_count(14,app_content,"app_last2")
    app_feature = pd.merge(app_feature,app_last2days_counts,on=['user_id'],how='left')
    del app_last2days_counts
    gc.collect()
    
    app_last3days_counts = get_last_day_count(13,app_content,"app_last3")
    app_feature = pd.merge(app_feature,app_last3days_counts,on=['user_id'],how='left')
    del app_last3days_counts
    gc.collect()
    
    app_last4days_counts = get_last_day_count(12,app_content,"app_last4")
    app_feature = pd.merge(app_feature,app_last4days_counts,on=['user_id'],how='left')
    del app_last4days_counts
    gc.collect()
    
    app_last5days_counts = get_last_day_count(11,app_content,"app_last5")
    app_feature = pd.merge(app_feature,app_last5days_counts,on=['user_id'],how='left')
    del app_last5days_counts
    gc.collect()
    
    app_last6days_counts = get_last_day_count(10,app_content,"app_last6")
    app_feature = pd.merge(app_feature,app_last6days_counts,on=['user_id'],how='left')
    del app_last6days_counts
    gc.collect()
    
    app_last7days_counts = get_last_day_count(9,app_content,"app_last7")
    app_feature = pd.merge(app_feature,app_last7days_counts,on=['user_id'],how='left')
    del app_last7days_counts
    gc.collect()
    	
    end = time.clock()
    print(info+"\b"*(len(info)+1))
    print('Running time: %s Seconds'%(end-start))
    
    return app_feature

time: 10.9 ms


In [23]:
def video_feature(beginday,endday):
    
    info = getMemCpu()
    start = time.clock()
    
    video_content = video[(video.day>=beginday)&(video.day<=endday)]
    video_content.day = video_content.day - beginday+1
    
    video_feature = day_feature('video_day_',video_content)
    
    video_nums=video_content.groupby('user_id')['day'].nunique().astype("float32").reset_index().rename(columns={'day':'video_nums'})
    video_feature = pd.merge(video_feature,video_nums,on=['user_id'],how='left')
    print("video_nums")
    del video_nums
    gc.collect()
    
    # video_last1days_counts = get_last_day_count(15,video_content,"video_last1")
    # app_feature = pd.merge(video_feature,video_last1days_counts,on=['user_id'],how='left')
    # del video_last1days_counts
    # gc.collect()
    
    # video_last2days_counts = get_last_day_count(14,video_content,"video_last2")
    # video_feature = pd.merge(video_feature,video_last2days_counts,on=['user_id'],how='left')
    # del video_last2days_counts
    # gc.collect()
    
    # video_last3days_counts = get_last_day_count(13,video_content,"video_last3")
    # video_feature = pd.merge(video_feature,video_last3days_counts,on=['user_id'],how='left')
    # del video_last3days_counts
    # gc.collect()
    
    # video_last4days_counts = get_last_day_count(12,video_content,"video_last4")
    # video_feature = pd.merge(video_feature,video_last4days_counts,on=['user_id'],how='left')
    # del video_last4days_counts
    # gc.collect()
    
    # video_last5days_counts = get_last_day_count(11,video_content,"video_last5")
    # video_feature = pd.merge(video_feature,video_last5days_counts,on=['user_id'],how='left')
    # del video_last5days_counts
    # gc.collect()
    
    # video_last6days_counts = get_last_day_count(10,video_content,"video_last6")
    # video_feature = pd.merge(video_feature,video_last6days_counts,on=['user_id'],how='left')
    # del video_last6days_counts
    # gc.collect()
    
    video_last7days_counts = get_last_day_count(9,video_content,"video_last7")
    video_feature = pd.merge(video_feature,video_last7days_counts,on=['user_id'],how='left')
    del video_last7days_counts
    gc.collect()
    
    end = time.clock()
    print(info+"\b"*(len(info)+1))
    print('Running time: %s Seconds'%(end-start))
    
    return video_feature

time: 5.54 ms


In [24]:
def reg_feature(beginday,endday):
    
    reg_content = user_reg[(user_reg.register_day<=endday)] 
    reg_content.register_day = endday - reg_content.register_day + 1
    
    reg_content["device_type_register_count"] = (reg_content.groupby(by=["device_type", "register_type"])["register_type"].transform("count")).astype(np.uint16)
    print("device_type_register_count")
    
    reg_content["device_type_count"] = (reg_content.groupby(by=["device_type"])["device_type"].transform("count")).astype(np.uint16)
    print("device_type_count")
    
    reg_content["device_type_register_rate"] = (reg_content["device_type_register_count"] / (reg_content["device_type_count"]+0.00001)).astype(np.float32)
    print("device_type_register_rate")
    return reg_content

time: 3.63 ms


In [30]:
def act_feature(beginday,endday):
    
    info = getMemCpu()
    start = time.clock()
    
    act_content = user_act[(user_act.day>=beginday)&(user_act.day<=endday)]
    act_content.day = act_content.day - beginday + 1
    
    act_feature = day_feature('act_day_',act_content)
    
    act_last1days_counts = get_last_day_count(15,act_content,"act_last1")
    act_feature = pd.merge(act_feature,act_last1days_counts,on=['user_id'],how='left')
    del act_last1days_counts
    gc.collect()
    
    act_last2days_counts = get_last_day_count(14,act_content,"act_last2")
    act_feature = pd.merge(act_feature,act_last2days_counts,on=['user_id'],how='left')
    del act_last2days_counts
    gc.collect()
    
    act_last3days_counts = get_last_day_count(13,act_content,"act_last3")
    act_feature = pd.merge(act_feature,act_last3days_counts,on=['user_id'],how='left')
    del act_last3days_counts
    gc.collect()
    
    act_last4days_counts = get_last_day_count(12,act_content,"act_last4")
    act_feature = pd.merge(act_feature,act_last4days_counts,on=['user_id'],how='left')
    del act_last4days_counts
    gc.collect()
    
    act_last5days_counts = get_last_day_count(11,act_content,"act_last5")
    act_feature = pd.merge(act_feature,act_last5days_counts,on=['user_id'],how='left')
    del act_last5days_counts
    gc.collect()
    
    act_last6days_counts = get_last_day_count(10,act_content,"act_last6")
    act_feature = pd.merge(act_feature,act_last6days_counts,on=['user_id'],how='left')
    del act_last6days_counts
    gc.collect()
    
    act_last7days_counts = get_last_day_count(9,act_content,"act_last7")
    act_feature = pd.merge(act_feature,act_last7days_counts,on=['user_id'],how='left')
    del act_last7days_counts
    gc.collect()
    
    act_nums=act_content.groupby('user_id')['day'].nunique().astype("float32").reset_index().rename(columns={'day':'act_nums'})
    act_feature = pd.merge(act_feature,act_nums,on=['user_id'],how='left')
    print("act_nums")
    del act_nums
    gc.collect()
    
    author_nums=act_content.groupby('user_id')['author_id'].nunique().astype("float32").reset_index().rename(columns={'author_id':'author_nums'})
    act_feature = pd.merge(act_feature,author_nums,on=['user_id'],how='left')
    print("author_nums")
    del author_nums
    gc.collect()
    
    act_video_nums=act_content.groupby('user_id')['video_id'].nunique().astype("float32").reset_index().rename(columns={'video_id':'act_video_nums'})
    act_feature = pd.merge(act_feature,act_video_nums,on=['user_id'],how='left')
    print("act_video_nums")
    del act_video_nums
    gc.collect()
    
    action_type_nums=act_content.groupby('user_id')['action_type'].nunique().astype("float32").reset_index().rename(columns={'action_type':'action_type_nums'})
    act_feature = pd.merge(act_feature,action_type_nums,on=['user_id'],how='left')
    print("action_type_nums")
    del action_type_nums
    gc.collect()
    # #每一个video有多少行为类型
    # every_video_types=act_content.groupby(['user_id','video_id'])['action_type'].nunique().astype("float32").reset_index().rename(columns={'action_type':'every_video_types'})
    # act_feature = pd.merge(act_feature,every_video_types,on=['user_id'],how='left')
    # print("every_video_types")
    # del every_video_types
    # gc.collect()
    
    page_nums=act_content.groupby('user_id')['page'].nunique().astype("float32").reset_index().rename(columns={'page':'page_nums'})
    act_feature = pd.merge(act_feature,page_nums,on=['user_id'],how='left')
    print("page_nums")
    del page_nums
    gc.collect()
    
    act_continue_std = act_content[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_continue_std).rename('act_continue_std').astype("float32").reset_index()
    act_feature = pd.merge(act_feature,act_continue_std,on=['user_id'],how='left')
    print("act_continue_std")
    del act_continue_std
    gc.collect()
    
    act_continue_mean = act_content[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_continue_mean).rename('act_continue_mean').astype("float32").reset_index()
    act_feature = pd.merge(act_feature,act_continue_mean,on=['user_id'],how='left')
    print("act_continue_mean")
    del act_continue_mean
    gc.collect()
    
    
    gp = act_content[['user_id','day']].groupby(['user_id'])['day'].unique().apply(get_last_gap).rename('act_last_gap').reset_index()
    act_feature = pd.merge(act_feature,gp,on=['user_id'],how='left')
    del gp
    gc.collect()
    
    # gc.collect()
    # diff = diff_feature('act_day_diff_',act_content)
    # act_feature = pd.merge(act_feature,diff,on=['user_id'],how='left')
    
    cal_page = cal_feature_counts("act_page",act_content,'page')
    act_feature = pd.merge(act_feature,cal_page,on=['user_id'],how='left')
    print("cal_page")
    del cal_page
    gc.collect()
    
    cal_act_type = cal_feature_counts("act_type",act_content,'action_type')
    act_feature = pd.merge(act_feature,cal_act_type,on=['user_id'],how='left')
    print("cal_act_type")
    del cal_act_type
    gc.collect()
    
    author_counts = []
    author_counts = act_content.groupby(['user_id'])['author_id'].agg({("act_author_counts","count")}).reset_index()
    act_feature = pd.merge(act_feature,author_counts,on=['user_id'],how='left')
    print("author_counts")
    del author_counts
    gc.collect()
    
    gc.collect()
    author_count = []
    authors = act_content.groupby(['user_id','author_id'])['author_id'].agg({("act_author_count","count")}).reset_index()
    aut_fun = {('act_author_mean','mean'),('act_author_std','std'),('act_author_count','count')}
    author_count = authors.groupby("user_id")["author_id"].agg(aut_fun).reset_index()
    act_feature = pd.merge(act_feature,author_count,on=['user_id'],how='left')
    print("author_count")
    del author_count,authors
    gc.collect()
    
    eachday_acts=act_content.groupby(['user_id','day'])['day'].agg({('eachday_acts','count')}).reset_index()
    act_everyday_fun={('act_daily_max','max')}
    acts_daily_action=eachday_acts.groupby('user_id')['eachday_acts'].agg(act_everyday_fun).reset_index()
    act_feature = pd.merge(act_feature,acts_daily_action,on=['user_id'],how='left')
    print("acts_daily_action")
    del acts_daily_action,eachday_acts
    gc.collect()
    
    #单个视频、作者最大行为数
    gc.collect()
    acts_single_video=act_content.groupby(['user_id','video_id'])['video_id'].agg({('acts_single_video','count')}).reset_index()
    act_single_video_fun={('single_video_max','max')}
    acts_single_video_action=acts_single_video.groupby('user_id')['acts_single_video'].agg(act_single_video_fun).reset_index()
    act_feature = pd.merge(act_feature,acts_single_video_action,on=['user_id'],how='left')
    print("acts_single_video_action")
    del acts_single_video,acts_single_video_action
    gc.collect()
    
    #同一个video的行为种类——>>平均每个video上的行为种类
    a_types=act_content.groupby(['user_id','video_id'])['action_type'].agg({('types_video','count')}).reset_index()
    act_type_fun={('every_video_type_mean','mean')}
    types_video_action=a_types.groupby('user_id')['types_video'].agg(act_type_fun).reset_index()
    act_feature = pd.merge(act_feature,types_video_action,on=['user_id'],how='left')
    print("types_video_action")
    del a_types,types_video_action
    gc.collect()
    
    #行为日志，每天浏览页面种类平均数
    page_types=act_content.groupby(['user_id','day'])['page'].agg({('page_types','count')}).reset_index()
    page_type_fun={('every_page_type_mean','mean')}
    every_page_type_action=page_types.groupby('user_id')['page_types'].agg(page_type_fun).reset_index()
    act_feature = pd.merge(act_feature,every_page_type_action,on=['user_id'],how='left')
    print("every_page_type_action")
    del page_types,every_page_type_action
    gc.collect()
    
    end = time.clock()
    print(info+"\b"*(len(info)+1))
    print('Running time: %s Seconds'%(end-start))
    return act_feature

time: 26.6 ms


In [32]:
def merge_feature(reg_feature,act_feature,video_feature,app_feature):

    #train_feature = []
    train_feature = reg_feature
    train_feature = pd.merge(train_feature,act_feature,on=['user_id'],how='left')
    train_feature = pd.merge(train_feature,video_feature,on=['user_id'],how='left')
    train_feature = pd.merge(train_feature,app_feature,on=['user_id'],how='left')
    return train_feature

time: 2.49 ms


In [33]:
def is_active(df,d1,d2):
    gp1 = app[(app.day>=d1) & (app.day<=d2)][['user_id']].groupby(['user_id']).size().rename('app').reset_index()
    gp2 = video[(video.day>=d1) & (video.day<=d2)][['user_id']].groupby(['user_id']).size().rename('video').reset_index()
    gp3 = user_act[(user_act.day>=d1) & (user_act.day<=d2)][['user_id']].groupby(['user_id']).size().rename('act').reset_index()
    
    df = df.merge(gp1,on=['user_id'],how='left')
    df = df.merge(gp2,on=['user_id'],how='left')
    df = df.merge(gp3,on=['user_id'],how='left')
    
    df['app'] = df['app'].fillna(0)
    df['video'] = df['video'].fillna(0)
    df['act'] = df['act'].fillna(0)
    df['count'] = df['app']+df['video']+df['act']
    
    def isnotzero(x):
        if x>0:
            return 1
        return 0
    y = df['count'].apply(isnotzero)
    return y  

time: 5.62 ms


In [28]:
mkdir /home/kesci/bigdata/train2

time: 238 ms


In [34]:
reg3 = reg_feature(15,30)
# reg3.to_csv("/home/kesci/bigdata/train3/reg3.csv",index = False)

act3 = act_feature(15,30)
# act3.to_csv("/home/kesci/bigdata/train3/act3.csv",index = False)

app3 = app_feature(15,30)
# app3.to_csv("/home/kesci/bigdata/train3/app3.csv",index = False)

video3 = video_feature(15,30)
# video3.to_csv("/home/kesci/bigdata/train3/video3.csv",index = False)

train3_feature = merge_feature(reg3,act3,video3,app3)
train3_feature.to_csv("/home/kesci/bigdata/train3/fuck3_feature.csv",index = False)

time: 45.2 s


In [35]:
del reg3,video3,act3,app3,train3_feature
gc.collect()
reg2 = reg_feature(8,23)
act2 = act_feature(8,23)
app2 = app_feature(8,23)
video2 = video_feature(8,23)
train2_feature = merge_feature(reg2,act2,video2,app2)
label2 = is_active(train2_feature,24,30)
train2_feature['label'] = label2
train2_feature.to_csv("/home/kesci/bigdata/train2/fuck2_feature.csv",index = False)
del reg2,video2,act2,app2,train2_feature
gc.collect()

/opt/conda/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

device_type_register_count
device_type_count
device_type_register_rate


/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


act_nums
author_nums
act_video_nums
action_type_nums
page_nums
act_continue_std
act_continue_mean
cal_page
cal_act_type
author_counts
author_count
acts_daily_action
acts_single_video_action
types_video_action
every_page_type_action
Memory usage:32%  CPU:25.50%
Running time: 1357.014658 Seconds
app_nums
Memory usage:42%  CPU:25.40%
Running time: 457.50469499999963 Seconds
video_nums
Memory usage:32%  CPU:25.40%
Running time: 15.265809999999874 Seconds


111

time: 31min 18s


In [36]:
reg1 = reg_feature(1,16)
act1 = act_feature(1,16)
app1 = app_feature(1,16)
video1 = video_feature(1,16)
train1_feature = merge_feature(reg1,act1,video1,app1)
label1 = is_active(train1_feature,17,23)
train1_feature['label'] = label1
train1_feature.to_csv("/home/kesci/bigdata/train1/fuck1_feature.csv",index = False)

device_type_register_count
device_type_count


/opt/conda/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

device_type_register_rate
act_nums
author_nums
act_video_nums
action_type_nums
page_nums
act_continue_std
act_continue_mean
cal_page
cal_act_type
author_counts
author_count
acts_daily_action
acts_single_video_action
types_video_action
every_page_type_action
Memory usage:32%  CPU:25.60%
Running time: 755.4405539999998 Seconds
app_nums
Memory usage:35%  CPU:25.70%
Running time: 319.94611800000075 Seconds
video_nums
Memory usage:32%  CPU:25.40%
Running time: 10.32046999999966 Seconds
time: 18min 38s
